In [6]:
import os

In [7]:
os.chdir('/Users/martaplonka/Documents/DeepLearning/MonkeysKaggle')

In [9]:
os.getcwd()

'/Users/martaplonka/Documents/DeepLearning/MonkeysKaggle'

In [14]:
from src.dataset import *

In [16]:
MonkeyDataset.MonkeyDataset('.data', transform=None, phase='training', download=True)